### Convalution_classify_CIFAR10

David 9回顾去年著名论文All you need is a good init，当时提出了一种新型初始化权重的方法，号称在Cifar-10上达到94.16%的精度，这里用keras还原一下试试。

运用tensorflow和高级api——keras，构建一个卷积神经网络模型。应用到数据集CIFAR10，作图片类别识别任务。

### import

先导入需要的包，包括常用数据处理和图表显示模块。  
需要下载初始化权重的包：git clone https://github.com/ducha-aiki/LSUV-keras  
可惜这个包里面有错误，需要略微修改。  
66至68行替换为如下代码：  
```
layer.set_weights(weights_and_biases)
weights_and_biases[1] /= np.sqrt(variance) / np.sqrt(needed_variance)
layer.set_weights(weights_and_biases)
```

In [1]:
import os
import tensorflow.compat.v1 as tf
from tensorflow.compat.v1 import keras
from tensorflow.compat.v1.keras import layers,datasets,utils,models
from tensorflow.compat.v1.keras.preprocessing.image import ImageDataGenerator
from tensorflow.compat.v1.keras.models import Sequential
from tensorflow.compat.v1.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.compat.v1.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalMaxPooling2D 
import numpy as np
from tensorflow.compat.v1.keras.utils import to_categorical
from lsuv_init import LSUVinit

import matplotlib.pyplot as plt
%matplotlib inline


config = tf.ConfigProto(allow_soft_placement=True)                   
config.gpu_options.per_process_gpu_memory_fraction = 0.8
tf.keras.backend.set_session(tf.Session(config=config))

In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.keras.backend.set_session(tf.Session(config=config))

### 数据加载

设定一下batch大小，类别数，训练轮数，还有就是启用数据增强。

In [3]:
batch_size = 16
num_classes = 10
epochs = 5
data_augmentation = True

使用的是keras内置加载数据。  
由于数据源是国外多伦多大学网站，这一步要事先下载好放到指定位置：  
windows：user/.keras/datasets/  
linux：~/.keras/datasets/  
并且对压缩包改名：  
cifar-10-python.tar.gz -> cifar-10-batch.tar.gz

In [4]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

对数据归一化、独热编码化

In [5]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_test /= 255.0
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

### 创建model

这是一个15层的CNN神经网络，具体结构很容易看明白。

In [6]:
def conv_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(48, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(48, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(80, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(80, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(80, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(80, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(80, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(128, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(GlobalMaxPooling2D())
    model.add(Dropout(0.25))
    
    model.add(Dense(500))
    model.add(Activation('relu'))
    model.add(Dropout(0.25))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    return model

### 定义损失函数

这一块没有特殊的地方，常规的损失函数定义和模型编译。

In [7]:
model = conv_model()

# initiate RMSprop optimizer
# opt = keras.optimizers.Adam(lr=0.0001)
opt = keras.optimizers.SGD(lr=0.0001)
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

### 特殊初始化

论文的重点特色就在于此，使用特殊的初始化方法，得以提升性能。  
第二行不重要，是监控。

In [8]:
model = LSUVinit(model,x_train[:batch_size,:,:,:]) 

conv2d
LSUV initializing conv2d
0.0176714
0.99999976
activation
conv2d_1
LSUV initializing conv2d_1
0.045270506
1.0
activation_1
conv2d_2
LSUV initializing conv2d_2
0.0128124105
0.9999999
activation_2
conv2d_3
LSUV initializing conv2d_3
0.034055423
0.9999998
activation_3
conv2d_4
LSUV initializing conv2d_4
0.03496846
1.0000001
activation_4
max_pooling2d
dropout
conv2d_5
LSUV initializing conv2d_5
0.031640578
0.9999998
activation_5
conv2d_6
LSUV initializing conv2d_6
0.01377221
0.9999999
activation_6
conv2d_7
LSUV initializing conv2d_7
0.023633204
0.9999999
activation_7
conv2d_8
LSUV initializing conv2d_8
0.015010685
0.9999998
activation_8
conv2d_9
LSUV initializing conv2d_9
0.015397631
1.0000001
activation_9
max_pooling2d_1
dropout_1
conv2d_10
LSUV initializing conv2d_10
0.012721574
1.0000001
activation_10
conv2d_11
LSUV initializing conv2d_11
0.009487288
0.9999999
activation_11
conv2d_12
LSUV initializing conv2d_12
0.010537763
1.0
activation_12
conv2d_13
LSUV initializing conv2d_13
0.

In [9]:
model.load_weights('./model/initmethods_weigths.h5')

In [10]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='.\\Graph2', histogram_freq=0, write_graph=True, write_images=True)

### 数据增强

数据增强，属于图像处理的常规手段了。  
可以选择不开启。

In [11]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True, callbacks=[tbCallBack])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    '''
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    '''
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)
    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test), callbacks=[tbCallBack])

Using real-time data augmentation.
Epoch 1/5
3125/3125 [==============================] - 168s 54ms/step - loss: 1.2419 - accuracy: 0.5519 - val_loss: 1.2221 - val_accuracy: 0.5564
Epoch 2/5
3125/3125 [==============================] - 169s 54ms/step - loss: 1.2422 - accuracy: 0.5535 - val_loss: 1.2279 - val_accuracy: 0.5547
Epoch 3/5
3125/3125 [==============================] - 172s 55ms/step - loss: 1.2377 - accuracy: 0.5530 - val_loss: 1.2127 - val_accuracy: 0.5596
Epoch 4/5
3125/3125 [==============================] - 165s 53ms/step - loss: 1.2423 - accuracy: 0.5528 - val_loss: 1.2088 - val_accuracy: 0.5613
Epoch 5/5
3125/3125 [==============================] - 171s 55ms/step - loss: 1.2335 - accuracy: 0.5543 - val_loss: 1.2195 - val_accuracy: 0.5570


In [12]:
model.save('CIFAR10_conv_model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: CIFAR10_conv_model\assets


In [13]:
model.save_weights('./model/initmethods_weigths.h5')

In [14]:
model2.load_weights('./weigths.h5')

NameError: name 'model2' is not defined